<a href="https://colab.research.google.com/github/AntonJachjajev/CNN/blob/main/CNN_Caffe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Melanoma detection using - Caffe

In [2]:
#Connect google colab to google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
import pandas as pd 

training_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Part3_GroundTruth.csv') 
testing_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv') 

training_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv')
testing_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv')

path_training_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/'
path_testing_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/'


#add new column to training_truth_data witht he path to corresponding image
path_to_training_images_array = []
for x in training_truth_data.values:
    path_to_image = path_training_images + x[0] +'.jpg'
    path_to_training_images_array.append(path_to_image)
training_truth_data['path_to_image'] = path_to_training_images_array
training_truth_data = pd.merge(training_truth_data, training_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)

#add new column to test_truth_data witht he path to corresponding image
path_to_test_images_array = []
for x in testing_truth_data.values:
  path_to_image = path_testing_images + str(x[0])+'.jpg'
  path_to_test_images_array.append(path_to_image)
print(path_to_test_images_array)
testing_truth_data['path_to_image'] = path_to_test_images_array
testing_truth_data = pd.merge(testing_truth_data, testing_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)


print(training_truth_data.head())
print(testing_truth_data.head())














['/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012086.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012092.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012095.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012134.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012136.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012147.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012149.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012152.jpg', '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC_0012178.jpg', '/content/drive/My